### Introduction
This Workflow simulates putting the E. Coli core genome under the regulation of specific transcription factors and mutating certain reactions. The programm then calculates the changed metabolic flux. The changes can be visualized with an *escher* metabolic map.

### Set up symstem

In [1]:
# changed or new functions for this workflow
%run "TFMetEngSim.ipynb" 

print('System ready')

System ready


### Set up wildtype and add transcription factor regulation
Set the name of the transcriptionfactor as string on the second variable of `add_TranscriptionFactor()` and set the RctIDs of the targeted enzymes as list of strings as the third variable. Add more rows with more regulators if needed.

In [5]:
wtHost = Ecol()
print('Wild type growth rate: {:.2f}'.format(wtHost.strain.objective))
WTdf = wtHost.strain.genes_df 

Wild type growth rate: 0.87


### Mutate the Host
Insert the RctIDs of the Reactions you want to mutate in `Mutants` as a list of strings. 
Insert the mutated Sequence in `insertSeq` as a string.
The sequence to be replaced is standardized as the always present ATTGA.

In [14]:
# Generating mutant (define the mutations by hand and delete the automatically generated genome for now)

myHost = deepcopy(wtHost)

MTdf = myHost.strain.genes_df

In [37]:
WTdf['Fluxes']= 2.0
WTdf
MTdf['Fluxes']= 1.0
MTdf

,RctID,Expression,Promoter,ORF,Fluxes,Expr2Flux
0,PFK,2.916,GCCCATTGACGCTGAATTTACAGCTGGATACTTTTGCACG,ATGGTCGAACCGTAA,1.0,2.564260
1,PFL,3.288,GCCCATTGACTGATTATCCATCAGGTAATACCTTTGCACG,ATGCTGTCTGAGTAA,1.0,0.000000
2,PGI,1.991,GCCCATTGACCGTCTCCACGCGAAGCCATATCGTTGCACG,ATGGGTCAGCTGTGA,1.0,2.441417
3,PGK,4.633,GCCCATTGACATAGTATATTCTACTCTATACCGTTGCACG,ATGCTCCCATACTGA,1.0,-3.458564
4,PGL,2.603,GCCCATTGAACATGCCAACGGGTAATTCTAACCTTGCACG,ATGGACTCCCGCTGA,1.0,1.905488
...,...,...,...,...,...,...
90,NADH16,1.807,GCCCATTGATCACGTGTTGCGGGGAGTATAACCTTGCACG,ATGTCCTCTCATTAG,1.0,21.325185
91,NADTRHD,1.141,GCCCATTGAGCCTCCCTAGGATAGCGGCTACTGTTGCACG,ATGAATCAATCATAG,1.0,0.000000
92,NH4t,1.266,GCCCATTGACGTATGTGGCGAGGCAGCATAGGGTTGCACG,ATGGAACGCCCATAA,1.0,3.764075
93,O2t,1.109,GCCCATTGATTACTCTTACGTATTCGTGTATGCTTGCACG,ATGATTGTTCTTTAA,1.0,19.656891


## Calculation of mutant flux distribution
The updated expression activities are converted to reference flux values and according to the reference flux new boundaries are set in the GSMM. The wild type and mutant flux distributions are stored in a dictionary for visualization in Escher.

In [38]:
# solution,_ = Help_FluxCalculator('Ecol', myHost.strain, AllNewDF)
RefFluxDict = WTdf.set_index('RctID')['Fluxes'].to_dict()
NewFluxDict = {a:b for a,b in zip(WTdf['RctID'].values, MTdf['Fluxes'].values)}
AllFluxDict = [RefFluxDict, NewFluxDict]
# {WTdf['RctID'].tolist(): solution}

### Visualization of flux differences
Escher is used to visualize the flux differences on the central carbon metabolism map. Open the output-html file `Escher-Metabol-Map.html` by double click in the Jupyter Lab environment and `Trust` the file.

Then load the out.json file in Data -> load reaction data

In [39]:
# name of the new file
filename = 'all2_1'

In [40]:
# create json file of AllFluxDict

import json

flux_dictionary = AllFluxDict
with open(filename+'.json', 'w') as f:
    json.dump(flux_dictionary, f)
    
print('created json file')

created json file


### Additional tools

In [ ]:
# load to excel file
AllNewDF.to_excel(filename+'.xlsx')

In [ ]:
# for visualizing all rows of a dataframe:
pd.set_option('display.max_rows', None)
# insert the name of the Dataframe:
#AllNewDF

In [ ]:
# test the strength of a promoter
a = Help_PromoterStrength('Ecol', 'GCCCATTCCCCTGGAAGCCGAAGGTGGTCGAGCGATATAATTGCACG')
a

##### Combinations of promoters and mutations:

Promoter: GCCCATTGACAAGGCTCTCGCGGCCAGGTATAATTGCAAA

* 5.537 --> 1.507:  'TAATTCCCC'

Promoter: GCCCATTGAGCCGAAGGTGGTCGAGCGATATAATTGCACG

* 1.7 --> 3.419:  'ATTCCCCTGGAA'
